# MACHINE LEARNING

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import glob


## 1. Module Machine Learning
* https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_ml/py_table_of_contents_ml/py_table_of_contents_ml.html
* https://docs.opencv.org/4.x/dd/ded/group__ml.html#ga9c57a2b823008dda53d2c7f7059a8710
* cv2.ml_KNearest()
* cv2.ml_SVM()
* cv2.ml_Dtrees()
* cv2.ml_LogisticRegression()
* cv2.kmeans()

In [11]:
dig = cv2.imread('DATA/raw/digits.png')
gray = cv2.cvtColor(dig,cv2.COLOR_BGR2GRAY)

# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()

# Initiate kNN, train the data, then test it with test data for k=1
knn = cv2.ml.KNearest_create()
knn.train(train,0,train_labels)
ret,result,neighbours,dist = knn.findNearest(test,k=5)

# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print(accuracy)

91.76


In [51]:
dig = cv2.imread('DATA/raw/digits.png')
gray = cv2.cvtColor(dig,cv2.COLOR_BGR2GRAY)

# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()

# Train the SVM
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
svm.train(train, cv2.ml.ROW_SAMPLE, train_labels)

_,result = svm.predict(test)

matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print(accuracy)

90.2


In [60]:
dig = cv2.imread('DATA/raw/digits.png')
gray = cv2.cvtColor(dig,cv2.COLOR_BGR2GRAY)

# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()

dt = cv2.ml.DTrees_create()
dt.setMaxDepth(7)
dt.setCVFolds(1)
dt.train(train, cv2.ml.DTREES_PREDICT_AUTO, train_labels)

_, result = dt.predict(test)

matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print(accuracy)

71.08


## 2. PCA and SVD

## 3. Module Deep Neural Network
* cv2.dnn.ClassificationModel()
* cv2.dnn.DetectionModel()
* cv2.dnn.SegmetaionModel()
* cv2.dnn.TextDetectionModel()
* cv2.dnn.TextRecognitionModel()
* cv2.dnn.blobFromImage()
* cv2.dnn.readNetFromDarknet()
* cv2.dnn.readNetFromTensorflow()
* cv2.dnn.readNetFromTorch()